### In this file, we will conduct all of our tests

In [22]:
import pandas as pd
import numpy as np
import torch
from dgl.dataloading.pytorch import GraphDataLoader
from tqdm.notebook import tqdm
import nbimporter
import Data_Structs as ds
import model as mfile
from score import test

In [2]:
# sample code from https://docs.dgl.ai/tutorials/blitz/3_message_passing.html#sphx-glr-tutorials-blitz-3-message-passing-py

In [3]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(200):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))



In [33]:
train_dataset = ds.SyntheticDataset()
train_dataloader = GraphDataLoader(train_dataset, batch_size = 16, shuffle = True)

In [12]:
i=0
for x, y in train_dataloader:
    print(i, x, y)
    i+=1

0 Graph(num_nodes=506, num_edges=1042,
      ndata_schemes={'atom_feats': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'bond_feats': Scheme(shape=(5,), dtype=torch.int64)}) tensor([-2947.7140, -3079.9667, -3850.3477, -4505.0814, -3151.7599, -3189.9621,
        -2953.7390, -3781.2509, -1979.5477, -3154.9586, -2285.4817, -2359.1612,
        -3573.3221, -4046.1658, -2598.5198, -3297.3016], dtype=torch.float64)
1 Graph(num_nodes=505, num_edges=1010,
      ndata_schemes={'atom_feats': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'bond_feats': Scheme(shape=(5,), dtype=torch.int64)}) tensor([-2584.1013, -1860.9434, -2657.3172, -2766.2821, -3342.5018, -2281.2007,
        -3910.2880, -3542.2311, -2879.6198, -2811.3324, -2758.8671, -2303.1349,
        -2888.3567, -2980.1870, -2599.5089, -3240.7047], dtype=torch.float64)
2 Graph(num_nodes=613, num_edges=1276,
      ndata_schemes={'atom_feats': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'bond

In [7]:
def train(model, epochs, debug_rate=100):
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    
    mse, running = 0, 0
    for epoch in tqdm(range(epochs)):
        
        count = 0
        for batch_x, batch_y in train_dataloader:
            optimizer.zero_grad()
            
            bf = batch_x.edata['bond_feats'].float()
            af = batch_x.ndata['atom_feats'].float()
            y_pred = model(batch_x, af, bf)
            
            mse = ((y_pred.reshape(1,-1) - batch_y)**2).sum()
            running += mse.item()
            mse.backward()
            optimizer.step()
            
            
#             if count % debug_rate == 0:
#                 print('Sample', count, 'Loss:', mse)
#                 printf('Sample %d/%d').format()
            
            count += 1
            print(count)
        
        return
        print('Epoch', epoch, 'Loss:', mse)

#### Create and Train Model

In [8]:
# All graphs in the list have the same scheme size, so pull the dimensions from the first
node_dim = train_dataset[0][0].ndata['atom_feats'].shape[1]
edge_dim = train_dataset[0][0].edata['bond_feats'].shape[1]
print("Dimensions:", node_dim, "(node),", edge_dim, "(edge)")

Dimensions: 11 (node), 5 (edge)


In [9]:
model = mfile.MPNN_Rec(node_dim, edge_dim, out_dim=1)

In [10]:
train(model, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
